In [1]:
import numpy as np
import pandas as pd
from arch import arch_model
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:

# Fit GARCH model function
def fit_garch(df, vol='Garch', p=1, q=1, dist='Normal'):
    # Calculate the log returns
    log_returns = np.log(df / df.shift(1)).dropna()

    # Fit the GARCH model
    model = arch_model(log_returns, vol=vol, p=p, q=q, dist=dist)
    results = model.fit(update_freq=0, disp='off')

    return results



In [3]:
# Load Data
df = pd.read_excel("Daily closes.xlsx", "UK Nat Gas 1st Month Daily")

In [4]:
df.tail()

,Date,FN1 Comdty - Last Price,FN1 Comdty - Open Interest,FN1 Comdty - Volume
2670,2012-12-14,67.11,23030.0,5945.0
2671,2012-12-13,66.70,24255.0,5760.0
2672,2012-12-12,66.91,23660.0,5200.0
2673,2012-12-11,66.77,24555.0,6300.0
2674,2012-12-10,67.37,23885.0,9685.0


In [5]:
drop_list = ["FN1 Comdty - Volume", "FN1 Comdty - Open Interest"]

In [6]:
df.drop(drop_list, inplace=True, axis=1)

In [7]:
df.set_index("Date", inplace=True)

In [8]:
df.tail()

,FN1 Comdty - Last Price
Date,
2012-12-14,67.11
2012-12-13,66.70
2012-12-12,66.91
2012-12-11,66.77
2012-12-10,67.37


In [9]:
df.tail()

,FN1 Comdty - Last Price
Date,
2012-12-14,67.11
2012-12-13,66.70
2012-12-12,66.91
2012-12-11,66.77
2012-12-10,67.37


In [10]:
# Fit a GARCH(1, 1) model
garch_results = fit_garch(df)

/Users/mus/opt/anaconda3/lib/python3.9/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002318. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [11]:
# Extract standardized residuals and conditional volatility
residuals = garch_results.resid
conditional_volatility = garch_results.conditional_volatility

# Combine the residuals and conditional volatility in a new DataFrame
plot_data = pd.DataFrame({'Standardized Residuals': residuals, 'Conditional Volatility': conditional_volatility})


In [12]:
plot_data.tail()

,Standardized Residuals,Conditional Volatility
Date,,
2012-12-14,0.006472,0.017368
2012-12-13,-0.006234,0.017030
2012-12-12,0.003038,0.016744
2012-12-11,-0.002200,0.016337
2012-12-10,0.008840,0.015985


In [13]:
def monthly_averages(df):
    # Ensure that the index is a DateTimeIndex
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)

    # Resample the DataFrame to a monthly frequency and calculate mean
    monthly_average_df = df.resample('M').mean()

    return monthly_average_df

# usage
df_monthly = monthly_averages(plot_data)

In [14]:
md=df_monthly.tail(24)

In [15]:
md["Conditional Volatility"].std()

0.03153994779560986

In [17]:
md.to_excel("NBP_monthly.xlsx")

In [ ]:
# Set Seaborn style
sns.set(style='whitegrid')

# Create subplots
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 8), sharex=True)

# Plot standardized residuals
sns.lineplot(data=plot_data['Standardized Residuals'], ax=axes[0], color='blue')
axes[0].set_title('Standardized Residuals')

# Plot conditional volatility
sns.lineplot(data=plot_data['Conditional Volatility'], ax=axes[1], color='red')
axes[1].set_title('Conditional Volatility')

# Display the plots
plt.tight_layout()
plt.savefig("Garch-NBP-Daily.png")